In [ ]:
!pip install -q datasets transformers

In [ ]:
# prompt: load the pruhtopia/rsc-5k-1 dataset in from huggingface

from datasets import load_dataset

dataset = load_dataset("pruhtopia/rsc-5k-1", split='train')

dataset = dataset.select(range(len(dataset) - 100))
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/308M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/303M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5349 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'text'],
    num_rows: 5249
})

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Load the processor and model
processor = BlipProcessor.from_pretrained("Gurveer05/blip-image-captioning-base-rscid-finetuned")
model = BlipForConditionalGeneration.from_pretrained("Gurveer05/blip-image-captioning-base-rscid-finetuned")

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], text=item["text"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4)

In [ ]:
from torch.cuda.amp import GradScaler, autocast
from transformers import get_scheduler

# Initialize optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        pixel_values = batch['pixel_values'].to(device)

        outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        if step % 100 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 0, Step 0, Loss: 0.12959839403629303
Epoch 0, Step 100, Loss: 0.05586658790707588
Epoch 0, Step 200, Loss: 0.05396576598286629
Epoch 0, Step 300, Loss: 0.06228630244731903
Epoch 0, Step 400, Loss: 0.06261120736598969
Epoch 0, Step 500, Loss: 0.05517217144370079
Epoch 0, Step 600, Loss: 0.059379201382398605
Epoch 0, Step 700, Loss: 0.05106358602643013
Epoch 0, Step 800, Loss: 0.04495066776871681
Epoch 0, Step 900, Loss: 0.03990413621068001
Epoch 0, Step 1000, Loss: 0.033730555325746536
Epoch 0, Step 1100, Loss: 0.05426947772502899
Epoch 0, Step 1200, Loss: 0.05163451284170151
Epoch 0, Step 1300, Loss: 0.03482728451490402
Epoch 1, Step 0, Loss: 0.04554232209920883
Epoch 1, Step 100, Loss: 0.053815487772226334
Epoch 1, Step 200, Loss: 0.04402650147676468
Epoch 1, Step 300, Loss: 0.04550638049840927
Epoch 1, Step 400, Loss: 0.04300915077328682
Epoch 1, Step 500, Loss: 0.051499996334314346
Epoch 1, Step 600, Loss: 0.052874572575092316
Epoch 1, Step 700, Loss: 0.04866626113653183
Epoch

In [ ]:
# prompt: push my trained model to the hub

import os
os.environ["HF_TOKEN"] = "_"

model.push_to_hub('pruhtopia/blip-rsc-5k-1')

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pruhtopia/blip-rsc-5k-1/commit/2f3c41205c9b0fd9a3cb9153b810c2763d0a6762', commit_message='Upload BlipForConditionalGeneration', commit_description='', oid='2f3c41205c9b0fd9a3cb9153b810c2763d0a6762', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pruhtopia/blip-rsc-5k-1', endpoint='https://huggingface.co', repo_type='model', repo_id='pruhtopia/blip-rsc-5k-1'), pr_revision=None, pr_num=None)